In [1]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_80fdb1cbcfba4ffc89754c5419936eaf = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-KpsSgZ6YTByw0VvYt62SM6470OAy9_yGH0B4gTdKFzL',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_80fdb1cbcfba4ffc89754c5419936eaf.get_object(Bucket='catsanddogs-donotdelete-pr-uve7xoh7zeegdm',Key='Book1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)



In [2]:
df_data_1.head()

,Postcode,Borough,Neighbourhood
0,M9Z,Not assigned,Not assigned
1,M9Y,Not assigned,Not assigned
2,M9X,Not assigned,Not assigned
3,M9W,Etobicoke,Northwest
4,M9V,Etobicoke,Albion Gardens


In [10]:

body = client_80fdb1cbcfba4ffc89754c5419936eaf.get_object(Bucket='catsanddogs-donotdelete-pr-uve7xoh7zeegdm',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2 = df_data_2.rename(columns={"Postal Code": "Postcode"})
df_data_2.head()


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
result = pd.merge(df_data_1, df_data_2, on='Postcode')


In [13]:
result.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9W,Etobicoke,Northwest,43.706748,-79.594054
1,M9V,Etobicoke,Albion Gardens,43.739416,-79.588437
2,M9V,Etobicoke,Beaumond Heights,43.739416,-79.588437
3,M9V,Etobicoke,Humbergate,43.739416,-79.588437
4,M9V,Etobicoke,Jamestown,43.739416,-79.588437
5,M9V,Etobicoke,Mount Olive,43.739416,-79.588437
6,M9V,Etobicoke,Silverstone,43.739416,-79.588437
7,M9V,Etobicoke,South Steeles,43.739416,-79.588437
8,M9V,Etobicoke,Thistletown,43.739416,-79.588437
9,M9R,Etobicoke,Kingsview Village,43.688905,-79.554724


In [17]:
df_filter = result.loc[result['Borough'].str.find("Toronto") != -1 ]
df_filter.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
49,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
51,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
52,M6S,West Toronto,Runnymede,43.651571,-79.484450
53,M6S,West Toronto,Swansea,43.651571,-79.484450
54,M6R,West Toronto,Parkdale,43.648960,-79.456325


In [19]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library



Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [22]:

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.653963, -79.387207.


In [32]:

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, df_filter in zip(df_filter['Latitude'], df_filter['Longitude'], df_filter['Borough'], df_filter['Neighbourhood']):
    label = '{}, {}'.format(df_filter, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [33]:
1


1